#### Hyperparameters


#### IMPORTS

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import tifffile as tif
import importlib
import os
import sys
import csv
from scipy.optimize import curve_fit
# local folder with homemade libraries
sys.path.append('/home/ngc/Documents/GitHub/multiview_utilities')
# HOMEMADE LIBRARIES AND IMPORTS
import math_utils as matut
import raw_images_manipulation_utilities as rim
importlib.reload(matut);
importlib.reload(rim); # reload the modules for updating to eventual changes

#### Local variables and constants.

Carefully read and change the variables in the next sectionto match the volume to be analysed.

In [8]:
SLICES = 69 # number of images for each volume
TIMEPOINTS =  200 # number of acquired volumes
TOTAL_IMAGES = TIMEPOINTS * SLICES # total of raw data-images
IMAGES_DIMENSION = 1024 # assumed square images.
# N.b. try to have 2^n pixels: it speeds up FFt calculations.
# Folder to get data and save data:
RAW_SLICES_FOLDER = '/home/ngc/Desktop/test_data/561_2um'
BACKGROUND_FOLDER = '/home/ngc/Desktop/test_data/561_2um'
VOLUMES_OUTPUT_FOLDER = '/home/ngc/Desktop/test_data/outputs'
PARAMS_OUTPUT_FOLDER = '/home/ngc/Desktop/test_data/params'

# good compromise to most situations, in case we want to skip the slow section "Feature extraction"
sigma = 20 

Create a list of names of all the volumes as 
**[timepoint (int), vol_cam_0 [str], vol_cam_1 (str)]**
next function uses a default file names paradigm, 
as saved by Labview software
(check arguments in "raw_images_manipulation_utilities" library)

In [9]:
data_list = rim.file_names_list(TIMEPOINTS)

#### Camera offset compensation

The cameras are not pixel-exactly aligned: the offset in their views
can be calculated with a 2D cross-correlation.
This process returns a 2D displacement vector, 
that will be addedd (during image processing) to CAM_01 
to have a better superpositoin with CAM_00.
(the opposite is also fine, subtracting from CAM_00)
In the next cell:

1. Select a volume, and in that volume a (few) slice(s) that will be used to find the 2D mismatch.

2. A pictorial result will tell if it worked. It should consist of: two superimposed imaged before displacement, after displacement, and a print() of the displacement vector

In [11]:
timepoint_for_offset_calculation = 0 # it should not have any impact

# select some slices to be used in the offset calculation.\
# Usually some central slices with a nice amount of information will do.
slices_for_offset_calculation = np.asarray((10, 30, 40, 60)) 
# Instead of using all the image (it can be absolutely done) 
# go quicker and use only a square central region of "extent" side
extent = 800

centre = IMAGES_DIMENSION / 2
shift_row, shift_col = 0, 0

# Open the stacks
stack_0 = rim.open_binary_stack(
    RAW_SLICES_FOLDER + data_list[timepoint_for_offset_calculation][1],
    BACKGROUND_FOLDER + '/Background_0.tif',
    size_x=IMAGES_DIMENSION,
    size_y=IMAGES_DIMENSION)
stack_1 = rim.open_binary_stack(
    RAW_SLICES_FOLDER + data_list[timepoint_for_offset_calculation][2],
    BACKGROUND_FOLDER + '/Background_1.tif',
    size_x=IMAGES_DIMENSION,
    size_y=IMAGES_DIMENSION)

for i in slices_for_offset_calculation:
    image_0 = stack_0[i, :, :]
    image_1 = stack_1[i, :, :]
    image_0_b = image_0[int(centre - extent/2):int(centre + extent/2),
                        int(centre - extent/2):int(centre + extent/2)]
    image_1_b = image_1[int(centre - extent/2):int(centre + extent/2),
                        int(centre - extent/2):int(centre + extent/2)]
    # n.b. images are one the mirror of the other, hence the [:, ::-1]
    shifts = rim.find_camera_registration_offset(image_0_b,
                            image_1_b[:, ::-1])
    shift_row += shifts[0] / len(slices_for_offset_calculation)
    shift_col += shifts[1] / len(slices_for_offset_calculation)

shift = np.asarray((shift_row, shift_col)).astype(np.int_)

If you want to see the result of the offset calculation:

In [12]:
%matplotlib qt

# choose any 2 slices
image_0 = stack_0[60, :, :]
image_1 = stack_1[60, :, :]
shifted_image = matut.shift_image(image_1[:,::-1], shift)

offset_recap = plt.figure('offset recap', figsize=(15, 7))
offset_recap.add_subplot(121)
plt.title('Before')
plt.imshow(image_0, alpha=.7, cmap='Oranges')
plt.imshow(image_1[:, ::-1], alpha=.5, cmap='Blues')
plt.xlabel('Pixels')
plt.ylabel('Pixels')
offset_recap.add_subplot(122)
plt.title('After')
plt.imshow(image_0, alpha=.7, cmap='Oranges')
plt.imshow(shifted_image, alpha=.5, cmap='Blues')
plt.xlabel('Pixels')
plt.show()
print(f'\nComputed shift: [', shift[0], ',', shift[1], '], ([row, cols] in pixels)\n')


Shift: [ 1 , 16 ], ([row, cols] in pixels)



#### Temporal (rigid) registration

Usually the sample suffers from a slow drift: it tends sinks.
In order to find the drift vector, an analysis similar to the previous
one is performed: a 3D cross-correlation between multiple couples
of volumes. 

Any volume can be selected (CAM_00 or CAM_01),
it will not impact the result. 
CAM_00 is selected by default.

The first is always the starting volume, the second is
as a function of time: it can be subtracted to each volume

In the next sequence:

1. Select the step [a.u., "timesteps"] used to include volumes
   in the algorithm.

2. A pictorial result can tell if it worked.
    It should consist of 3 linear fits for the 3 displacement compontents.
    Only one component is expected to be substantially != 0.
    If it does not work: use smaller timestep.
    If it still does not work: check the volumes "by eye", there must be something strange going on.

In [14]:
timestep = 50 # analysis performed every "timestep" volumes
extent_x_y = 800 # use only a central region of "extent_x_y" pixels
centre = IMAGES_DIMENSION / 2
shifts_row, shifts_col, shifts_plane = [], [], []

# by default we use camera 0, but using camera 1 is also fine.
# (just change filename and Background accordingly)
start_stack = rim.open_binary_stack(
    RAW_SLICES_FOLDER + data_list[0][1],
    BACKGROUND_FOLDER + '/Background_0.tif',
    size_x=IMAGES_DIMENSION,
    size_y=IMAGES_DIMENSION)

start_focus = start_stack[
    :,
    int(centre - extent_x_y/2):int(centre + extent_x_y/2),
    int(centre - extent_x_y/2):int(centre + extent_x_y/2)
    ]

for t in range(0, TIMEPOINTS, timestep):
    
    moving_stack = rim.open_binary_stack(
    RAW_SLICES_FOLDER + data_list[t][1],
    BACKGROUND_FOLDER + '/Background_0.tif',
    size_x=IMAGES_DIMENSION,
    size_y=IMAGES_DIMENSION)
    
    # Use only a central region (in the x-y plane)
    moving_focus = moving_stack[
    :,
    int(centre - extent_x_y/2):int(centre + extent_x_y/2),
    int(centre - extent_x_y/2):int(centre + extent_x_y/2)
    ]
    shift = rim.find_sample_drift(start_focus, moving_focus)
    # Save the shift vector
    shifts_plane.append(shift[0])
    shifts_row.append(shift[1])
    shifts_col.append(shift[2])

#### Fitting

In [15]:
# y = p x
def fit_simple(x, p):
    return p * x
# forcing the first point to be 0
shifts_plane[0] = 0
shifts_row[0] = 0
shifts_col[0] = 0

# STRONG LINEARITY ASSUMPTION
# the starting slope used during the fit is simply
# "(final_y - starting_y) / (initial_x - starting_x)"
strong_linear_plane = shifts_plane[-1] / len(shifts_plane)
strong_linear_row = shifts_row[-1] / len(shifts_plane)
strong_linear_col = shifts_col[-1] / len(shifts_plane)

x_plane = np.arange(0, len(shifts_plane), 1)
x_row = np.arange(0, len(shifts_row), 1)
x_col = np.arange(0, len(shifts_col), 1)

fit_plane_0 = curve_fit(
    fit_simple,
    x_plane,
    shifts_plane,
    p0=strong_linear_plane
    )
fit_row_0 = curve_fit(
    fit_simple,
    x_row,
    shifts_row,
    p0=strong_linear_row
    )
fit_col_0 = curve_fit(
    fit_simple,
    x_col,
    shifts_col,
    p0=strong_linear_col
    )

#### Plots

To summarize the temporal drift.
Errorbars cover:
- +/- 1 z-step for the depth
- +/- 1 pixel for x/y drift.

In [16]:
summary = plt.figure('Temporal Drift Recap')

summary.add_subplot(311)
plt.title('Depth drift')
plt.errorbar(x_plane, shifts_plane, yerr=1, fmt='o', color='dodgerblue')
y_plane_0 = fit_plane_0[0] * x_plane
plt.plot(x_plane, y_plane_0, '-', color='blue', alpha=.6)

summary.add_subplot(312)
plt.title('Vertical drift')
plt.errorbar(x_row, shifts_row, yerr=1, fmt='o', color='orange')
y_row_0 = fit_row_0[0] * x_row
plt.plot(x_row, y_row_0, '-', color='red', alpha=.6)

summary.add_subplot(313)
plt.title('Horizontal drift')
plt.errorbar(x_col, shifts_col, yerr=1, fmt='o', color='lightgreen')
y_col_0 = fit_col_0[0] * x_col
plt.plot(x_col, y_col_0, '-', color='green', alpha=.6)

plt.tight_layout()
plt.show()

# And print out the effective values that will be used 
# for the volumes processing
print(f'\nDepth drift: y = {fit_plane_0[0]} * x')   
print(f'With only first and last points would give: {strong_linear_plane}\n')
print(f'Vertical drift: y = {fit_row_0[0]} * x')   
print(f'With only first and last points would give: {strong_linear_row}\n')
print(f'Horizontal drift: y = {fit_col_0[0]} * x')   
print(f'With only first and last points would give: {strong_linear_col}\n')


Depth drift: y = [0.21428571] * x
With only first and last points would give: 0.125

Vertical drift: y = [-1.64285714] * x
With only first and last points would give: -1.25

Horizontal drift: y = [-1.78571429] * x
With only first and last points would give: -1.25



 ### Feature extraction
To merge images following Preibish et al. approximation to local entropy,
two sigmas must be selected. The first one should be larger than the linear
dimension of sensible features.
e.g. feature = 10 pixels, value = 15 pixels (???)
Find it with the cursos over the image,
and then save it in "sigma" value.


This may take a longer time then the rest.
It is in general a non essential part.

When merging, each view must contribute to the final image proportianally to the local amount of information it contains.
To measure this *local amount of information* various methods can be used. All give similar results.
Here a method from Preibish et al. is used.([article](https://www.spiedigitallibrary.org/conference-proceedings-of-spie/6914/1/Mosaicing-of-single-plane-illumination-microscopy-images-using-groupwise-registration/10.1117/12.770893.full?SSO=1))
The local information is approximated by the local standar deviation of each pixel. The definition of "local" is somehow arbitrary. For every pixel we look at a nerghborhood of 20 pixels. In general the local region extent can be tuned to maximaze the difference between the two images. The rationale is the following: when the two images are considered as bringing the same amount of information, a simple average is performed. If one image is much more informative than the other (in a particular region) then it should be condered **much more** than the other.

In practice, the local standard deviation is approximated by taking the difference between the original image and a low-pass filtered verion of itself: the extent of the gaussian employed in the filtering operatin determines the extent of what is considered *local*.

In [22]:
# Choose a volume
timepoint = 0

# Open the stacks
stack_0 = rim.open_binary_stack(
    RAW_SLICES_FOLDER + data_list[timepoint][1],
    BACKGROUND_FOLDER + '/Background_0.tif',
    size_x=IMAGES_DIMENSION,
    size_y=IMAGES_DIMENSION)
stack_1 = rim.open_binary_stack(
    RAW_SLICES_FOLDER + data_list[timepoint][2],
    BACKGROUND_FOLDER + '/Background_1.tif',
    size_x=IMAGES_DIMENSION,
    size_y=IMAGES_DIMENSION)

diff = []
# maximum extent of the local region
exploration_range = 70

# Choose an image for offset calculation
selected_slice = 40

for i in range(2, exploration_range):
    # define the low-pass filtering gaussian
    Gaus_1 = matut.gaussian_2D(stack_0.shape[1], [0,0], i+1)
    Gaus_1 = matut.FT2(Gaus_1)
    # local information evaluation
    w_1 = np.abs(stack_0[selected_slice, :, :] \
        - matut.IFT2((matut.FT2(stack_0[selected_slice, :, :]) * Gaus_1)))**2
    w_2 = np.abs(stack_1[selected_slice, :, :] \
        - matut.IFT2((matut.FT2(stack_0[selected_slice, :, ::-1]) * Gaus_1)))**2
    # evaluate the difference between normalized local weights
    diff.append(np.sum(np.abs(w_1/np.amax(w_1+w_2) - w_2/np.amax(w_2+w_1))))

sigma = np.argmax(diff) + 1

#### Check the results

In [26]:
%matplotlib qt
plt.figure('Find the sigma: local information comparison')
plt.plot(np.arange(len(diff)), diff/np.amax(diff), '.--')
plt.xlabel('Local extend (side of square ROI) [pixels]')
plt.ylabel('Local information difference [a.u.]')
plt.grid()
plt.show()

Saving all the hyperparameters in a .csv
- cameras offset
- drift fit parameters
- sigma 

In [27]:
with open(
    PARAMS_OUTPUT_FOLDER + '/hyperparameters.csv',
    'w',#
    newline='') as file:
    writer = csv.writer(file, delimiter=',')
    writer.writerow(["Parameter", "Value"])
    writer.writerow(["Camera vertical offset", 1])
    writer.writerow(["Camera horizontal offset", 1])
    writer.writerow(["T drift - depth", np.round(fit_plane_0[0][0], 2)])
    writer.writerow(["T drift - vertical", np.round(fit_row_0[0][0], 2)])
    writer.writerow(["T drift - horizontal", np.round(fit_col_0[0][0], 2)])
    writer.writerow(["sigma", np.round(sigma, 2)])